In [11]:
import pandas as pd

df = pd.read_csv("IMDB Dataset.csv.zip")
print(df.head())
print(df['sentiment'].value_counts())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


In [12]:
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer= PorterStemmer()
english_stopwords = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r"<.*?>","",text)
    text = re.sub(r'[^a-zA-Z]', "", text).lower()
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in english_stopwords]
    cleaned_text = "".join(words)
    return cleaned_text
df['cleaned_review'] = df['review'].apply(clean_text)
print(df[['review', 'cleaned_review']].head())

                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                      cleaned_review  
0  oneoftheotherreviewershasmentionedthatafterwat...  
1  awonderfullittleproductionthefilmingtechniquei...  
2  ithoughtthiswasawonderfulwaytospendtimeonatooh...  
3  basicallytheresafamilywherealittleboyjakethink...  
4  pettermatteisloveinthetimeofmoneyisavisuallyst...  


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
X = df['cleaned_review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Model Accuracy: 0.5007

Classification Report:
              precision    recall  f1-score   support

    negative       0.50      1.00      0.67      5000
    positive       1.00      0.00      0.00      5000

    accuracy                           0.50     10000
   macro avg       0.75      0.50      0.33     10000
weighted avg       0.75      0.50      0.33     10000

